### Creation of graphs that show the average number of transactions by hours for each day in a week (for determining queue at gas stations)

In [354]:
import pandas as pd 
import psycopg2 
import warnings
import numpy as np 
from sklearn.utils import shuffle
import cx_Oracle
import datetime
from pandas. tseries.offsets import DateOffset
from datetime import date
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import itertools

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

# Сбор данных за день недели (сумма кол-ва чеков по часам)
def df (month_09, month_10, month_11, azs):
	df = f"""select"""
	for i in t:
		if i == '23':
			df += f"""
count(distinct case when org_ksss in ({str(azs)[1:-1]}) and
	  (date_part('month', dt) = '09' and date_part('day', dt) in ({str(month_09)[1:-1]})
	    or
	   date_part('month', dt) = '10' and date_part('day', dt) in ({str(month_10)[1:-1]})	
	 	or
	   date_part('month', dt) = '11' and date_part('day', dt) in ({str(month_11)[1:-1]}))								 
	   and date_part('hour', operation_date) = '{i}' 
	   and ltr_vol<>0 and qnt = 0 then cheque_id else null end) as "{i}"
from public.pet_chk_22 """
		else:
			df += f"""
count(distinct case when org_ksss in ({str(azs)[1:-1]}) and
	  (date_part('month', dt) = '09' and date_part('day', dt) in ({str(month_09)[1:-1]})
	    or
	   date_part('month', dt) = '10' and date_part('day', dt) in ({str(month_10)[1:-1]})	
	 	or
	   date_part('month', dt) = '11' and date_part('day', dt) in ({str(month_11)[1:-1]}))								 
	   and date_part('hour', operation_date) = '{i}' 
	   and ltr_vol<>0 and qnt = 0 then cheque_id else null end) as "{i}",
       """
	return df

# перевод таблицы в формат: час:среднее кол-во чеков за час
def format (df):
    df = df.T
    df = df.reset_index()
    df.columns = ['time', 'amount']
    df['amount'] = df['amount']/13
    return df

# высчитывание среднего чека по кол-ву чеков на утро(23:00-05:59) и день(06:00-22:59)
def morning (df):
    morning = df[df.time.isin(['23','00','01','02','03','04','05'])].amount.sum()/7
    return morning
def day (df):
    day = df[~df.time.isin(['23','00','01','02','03','04','05'])].amount.sum()/17
    return day

# Создание графика
def chart (x, y, name, morning, day):

    fig, ax = plt.subplots()
    ax.bar(x, y, width=0.98)

    ax.set_ylabel('Среднее кол-во чеков', fontsize=14)
    ax.set_xlabel('Время, часы', fontsize=14)
    ax.set_title(name, fontsize=16)
    ax.grid(color='gray', linewidth=1, axis='y', alpha=0.6)


    fig.set_figwidth(20)  
    fig.set_figheight(8)

        # Добавление линий для утра и дня
    plt.axhline(y=morning, xmax=0.27, color='blue',  ls='dotted', linewidth=2, label = 'утро (23:00-05:59)')
    plt.axhline(y=morning, xmin=0.92, color='blue',  ls='dotted', linewidth=2)
    plt.legend(loc = 'upper left')
    plt.axhline(y=day, xmin=0.27, xmax=0.92, color='red', ls='dotted', linewidth=2, label = 'день (06:00-22:59)')
    plt.legend(loc = 'upper left')

    plt.show()

# время
t = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']
# Сбор данных за каждый день (сумма кол-ва чеков за каждый день по часам)
def df_full (month_09, month_10, month_11, azs):
    df_full = f"""select"""
    for s in month_09:
        for i in t:
             df_full += f"""
    count(distinct case when org_ksss in ({str(azs)[1:-1]}) and
        date_part('month', dt) = '09' and date_part('day', dt) = '{s}'
        and date_part('hour', operation_date) = '{i}' 
        and ltr_vol<>0 and qnt = 0 then cheque_id else null end) as "2022-09-{s} {i}:00:00", """
    for o in month_10:
        for i in t:
            df_full += f"""
    count(distinct case when org_ksss in ({str(azs)[1:-1]}) and
        date_part('month', dt) = '10' and date_part('day', dt) = '{o}'
        and date_part('hour', operation_date) = '{i}' 
        and ltr_vol<>0 and qnt = 0 then cheque_id else null end) as "2022-10-{o} {i}:00:00", """
    for n in month_11:
        for i in t:
            if n == month_11[-1] and i == '23':
                df_full += f"""
    count(distinct case when org_ksss in ({str(azs)[1:-1]}) and
        date_part('month', dt) = '11' and date_part('day', dt) = '{n}'
        and date_part('hour', operation_date) = '{i}' 
        and ltr_vol<>0 and qnt = 0 then cheque_id else null end) as "2022-11-{n} {i}:00:00"
        from public.pet_chk_22"""
            else:
                df_full += f"""
    count(distinct case when org_ksss in ({str(azs)[1:-1]}) and
        date_part('month', dt) = '11' and date_part('day', dt) = '{n}'
        and date_part('hour', operation_date) = '{i}' 
        and ltr_vol<>0 and qnt = 0 then cheque_id else null end) as "2022-11-{n} {i}:00:00", """
    return df_full

# создание колонки час
def hr_func(ts):
    return ts.hour

# перевод таблицы в формат: дата,время :среднее кол-во чеков за час: час 
def format_2 (df):
    df = df.T
    df = df.reset_index()
    df.columns = ['time', 'amount']
    df['time'] = pd.to_datetime(df['time'])
    # добавление колонки час
    df['hour'] = df['time'].apply(hr_func)
    return df

# Аномалии (кол-во чеков) УТРО
def morning_an (df):
    morning_full = df[df.hour.isin([23, 0, 1, 2, 3, 4, 5])].amount
    sort_morning_full = np.sort(morning_full) # сортировка собранных данных
    # высчитывание квартилей, нахождение верхних и нижних пределов
    Q1_morning=np.percentile(sort_morning_full, 25, interpolation='midpoint')
    Q2_morning=np.percentile(sort_morning_full, 50, interpolation='midpoint')
    Q3_morning=np.percentile(sort_morning_full, 75, interpolation='midpoint')
    IQR_morning = Q3_morning - Q1_morning
    low_lim_morning = Q1_morning - 1.5*IQR_morning
    up_lim_morning = Q3_morning + 1.5*IQR_morning
    # нахождение outliers
    outliers_morning_am=[]
    for x in morning_full:
        if ((x>up_lim_morning) or (x<low_lim_morning)):
            outliers_morning_am.append(x)
    outliers_morning = df[df.amount.isin(outliers_morning_am)]
    outliers_morning = outliers_morning.sort_values(by = ['hour'])
    return print('УТРО', outliers_morning), sns.boxplot(morning_full)

# Аномалии (кол-во чеков) ДЕНЬ
def day_an (df):
    day_full = df[~df.hour.isin([23, 0, 1, 2, 3, 4, 5])].amount
    sort_day_full = np.sort(day_full) # сортировка собранных данных
    # высчитывание квартилей, нахождение верхних и нижних пределов
    Q1_day=np.percentile(sort_day_full, 25, interpolation='midpoint')
    Q2_day=np.percentile(sort_day_full, 50, interpolation='midpoint')
    Q3_day=np.percentile(sort_day_full, 75, interpolation='midpoint')
    IQR_day = Q3_day - Q1_day
    low_lim_day = Q1_day - 1.5*IQR_day
    up_lim_day = Q3_day + 1.5*IQR_day
    # нахождение outliers
    outliers_day_am=[]
    for x in day_full:
        if ((x>up_lim_day) or (x<low_lim_day)):
            outliers_day_am.append(x)
    outliers_day = df[df.amount.isin(outliers_day_am)]
    outliers_day = outliers_day.sort_values(by = ['hour'])
    return print('ДЕНЬ', outliers_day), sns.boxplot(day_full)

In [268]:
# Сбор данных из Greenplum: Топ 50 АЗС по выручке
top_50 = f""" 
select org_ksss, org_name, npo,  sum(amount) as summa
from public.pet_chk_22
where date_part('month', dt) = '09' or date_part('month', dt) = '10' or date_part('month', dt) = '11'
group by org_ksss, org_name, npo
order by summa desc
limit 50
"""
#Считываем данные из sql
df_top = pd.read_sql(top_50, connection)

In [1]:
# Считываю 3 первые АЗС и 3 последние АЗС
first1 = df_top.head(3)
last1 = df_top.tail(3)
first = list(first1['org_ksss'])
last = list(last1['org_ksss'])
first = [str(i) for i in first]
last = [str(i) for i in last]

# ПОНЕДЕЛЬНИК

## Среднее кол-во чеков по часам в ПОНЕДЕЛЬНИК

In [342]:
# понедельники за каждый месяц
mon_09 = ['05','12','19','26']
mon_10 = ['03','10','17','24','31']
mon_11 = ['07','14','21','28']

### Первые 3 АЗС

In [344]:
# Сбор данных за понедельники (сумма кол-ва чеков по часам)
first_mon = pd.read_sql(df(mon_09, mon_10, mon_11, first), connection)
first_mon = format(first_mon)

In [2]:
chart(first_mon['time'], first_mon['amount'], 'Среднее кол-во чеков по часам в понедельник', morning(first_mon), day(first_mon))

### Последние 3 АЗС

In [346]:
# Сбор данных за понедельники (сумма кол-ва чеков по часам)
last_mon = pd.read_sql(df(mon_09, mon_10, mon_11, last), connection)
last_mon = format(last_mon)

In [3]:
chart(last_mon['time'], last_mon['amount'], 'Среднее кол-во чеков по часам в понедельник', morning(last_mon), day(last_mon))

## Аномалии (кол-во чеков) в ПОНЕДЕЛЬНИК

### Первые 3 АЗС

In [355]:
# создание таблицы (день:кол-во чеков:час) по каждому понедельнику
first_mon_full = pd.read_sql(df_full(mon_09, mon_10, mon_11, first), connection)
first_mon_full = format_2(first_mon_full)

In [7]:
morning_an (first_mon_full)

In [4]:
day_an(first_mon_full)

### Последние 3 АЗС

In [358]:
# создание таблицы (день:кол-во чеков:час) по каждому понедельнику
last_mon_full = pd.read_sql(df_full(mon_09, mon_10, mon_11, last), connection)
last_mon_full = format_2(last_mon_full)

In [8]:
morning_an (last_mon_full)

In [9]:
day_an(last_mon_full)

# ВТОРНИК

## Среднее кол-во чеков по часам во ВТОРНИК

In [361]:
# вторники за каждый месяц
tue_09 = ['06','13','20','27']
tue_10 = ['04','11','18','25']
tue_11 = ['01','08','15','22','29']

### Первые 3 АЗС

In [362]:
# Сбор данных за вторники (сумма кол-ва чеков по часам)
first_tue = pd.read_sql(df(tue_09, tue_10, tue_11, first), connection)
first_tue = format(first_tue)

In [10]:
chart(first_tue['time'], first_tue['amount'], 'Среднее кол-во чеков по часам во вторник', morning(first_tue), day(first_tue))

### Последние 3 АЗС

In [364]:
# Сбор данных за вторники (сумма кол-ва чеков по часам)
last_tue = pd.read_sql(df(tue_09, tue_10, tue_11, last), connection)
last_tue = format(last_tue)

In [11]:
chart(last_tue['time'], last_tue['amount'], 'Среднее кол-во чеков по часам во вторник', morning(last_tue), day(last_tue))

## Аномалии (кол-во чеков) во ВТОРНИК

### Первые 3 АЗС

In [366]:
# создание таблицы (день:кол-во чеков:час) по каждому вторнику
first_tue_full = pd.read_sql(df_full(tue_09, tue_10, tue_11, first), connection)
first_tue_full = format_2(first_tue_full)

In [12]:
morning_an (first_tue_full)

In [13]:
day_an (first_tue_full)

### Последние 3 АЗС

In [369]:
# создание таблицы (день:кол-во чеков:час) по каждому вторнику
last_tue_full = pd.read_sql(df_full(tue_09, tue_10, tue_11, last), connection)
last_tue_full = format_2(last_tue_full)

In [14]:
morning_an (last_tue_full)

In [15]:
day_an (last_tue_full)

# СРЕДА

## Среднее кол-во чеков по часам в СРЕДУ

In [372]:
# среды за каждый месяц
wed_09 = ['07','14','21','28']
wed_10 = ['05','12','19','26']
wed_11 = ['02','09','16','23','30']

### Первые 3 АЗС

In [373]:
# Сбор данных за среды (сумма кол-ва чеков по часам)
first_wed = pd.read_sql(df(wed_09, wed_10, wed_11, first), connection)
first_wed = format(first_wed)

In [374]:

df_temp = first_wed[first_wed.time.isin(['23','00','01','02','03','04','05'])]

In [16]:
df_temp.amount.mean()

In [17]:
chart(first_wed['time'], first_wed['amount'], 'Среднее кол-во чеков по часам в среду', morning(first_wed), day(first_wed))

### Последние 3 АЗС

In [378]:
# Сбор данных за среды (сумма кол-ва чеков по часам)
last_wed = pd.read_sql(df(wed_09, wed_10, wed_11, last), connection)
last_wed = format(last_wed)

In [18]:
chart(last_wed['time'], last_wed['amount'], 'Среднее кол-во чеков по часам в среду', morning(last_wed), day(last_wed))

## Аномалии (кол-во чеков) в СРЕДУ

### Первые 3 АЗС

In [380]:
# создание таблицы (день:кол-во чеков:час) по каждой среде
first_wed_full = pd.read_sql(df_full(wed_09, wed_10, wed_11, first), connection)
first_wed_full = format_2(first_wed_full)

In [19]:
morning_an (first_wed_full)

In [20]:
day_an (first_wed_full)

### Последние 3 АЗС

In [383]:
# создание таблицы (день:кол-во чеков:час) по каждой среде
last_wed_full = pd.read_sql(df_full(wed_09, wed_10, wed_11, last), connection)
last_wed_full = format_2(last_wed_full)

In [21]:
morning_an (last_wed_full)

In [22]:
day_an (last_wed_full)

# ЧЕТВЕРГ

## Среднее кол-во чеков по часам в ЧЕТВЕРГ

In [386]:
# четверга за каждый месяц
thu_09 = ['01','08','15','22','29']
thu_10 = ['06','13','20','27']
thu_11 = ['03','10','17','24']

### Первые 3 АЗС

In [387]:
# Сбор данных за четверга (сумма кол-ва чеков по часам)
first_thu = pd.read_sql(df(thu_09, thu_10, thu_11, first), connection)
first_thu = format(first_thu)

In [23]:
chart(first_thu['time'], first_thu['amount'], 'Среднее кол-во чеков по часам в четверг', morning(first_thu), day(first_thu))

### Последние 3 АЗС

In [389]:
# Сбор данных за четверга (сумма кол-ва чеков по часам)
last_thu = pd.read_sql(df(thu_09, thu_10, thu_11, last), connection)
last_thu = format(last_thu)

In [24]:
chart(last_thu['time'], last_thu['amount'], 'Среднее кол-во чеков по часам в четверг', morning(last_thu), day(last_thu))

## Аномалии (кол-во чеков) в ЧЕТВЕРГ

### Первые 3 АЗС

In [391]:
# создание таблицы (день:кол-во чеков:час) по каждому четвергу
first_thu_full = pd.read_sql(df_full(thu_09, thu_10, thu_11, first), connection)
first_thu_full = format_2(first_thu_full)

In [25]:
morning_an (first_thu_full)

In [26]:
day_an (first_thu_full)

### Последние 3 АЗС

In [394]:
# создание таблицы (день:кол-во чеков:час) по каждому четвергу
last_thu_full = pd.read_sql(df_full(thu_09, thu_10, thu_11, last), connection)
last_thu_full = format_2(last_thu_full)

In [27]:
morning_an (last_thu_full)

In [28]:
day_an (last_thu_full)

# ПЯТНИЦА

## Среднее кол-во чеков по часам в ПЯТНИЦУ

In [397]:
# пятницы за каждый месяц
fri_09 = ['02','09','16','23','30']
fri_10 = ['07','14','21','28']
fri_11 = ['04','11','18','25']

### Первые 3 АЗС

In [398]:
# Сбор данных за пятницы (сумма кол-ва чеков по часам)
first_fri = pd.read_sql(df(fri_09, fri_10, fri_11, first), connection)
first_fri = format(first_fri)

In [29]:
chart(first_fri['time'], first_fri['amount'], 'Среднее кол-во чеков по часам в пятницу', morning(first_fri), day(first_fri))

### Последние 3 АЗС

In [400]:
# Сбор данных за пятницы (сумма кол-ва чеков по часам)
last_fri = pd.read_sql(df(fri_09, fri_10, fri_11, last), connection)
last_fri = format(last_fri)

In [30]:
chart(last_fri['time'], last_fri['amount'], 'Среднее кол-во чеков по часам в пятницу', morning(last_fri), day(last_fri))

## Аномалии (кол-во чеков) в ПЯТНИЦУ

### Первые 3 АЗС

In [402]:
# создание таблицы (день:кол-во чеков:час) по каждой пятнице
first_fri_full = pd.read_sql(df_full(fri_09, fri_10, fri_11, first), connection)
first_fri_full = format_2(first_fri_full)

In [31]:
morning_an (first_fri_full)

In [32]:
day_an (first_fri_full)

### Последние 3 АЗС

In [405]:
# создание таблицы (день:кол-во чеков:час) по каждой пятнице
last_fri_full = pd.read_sql(df_full(fri_09, fri_10, fri_11, last), connection)
last_fri_full = format_2(last_fri_full)

In [33]:
morning_an (last_fri_full)

In [34]:
day_an (last_fri_full)

# СУББОТА

## Среднее кол-во чеков по часам в СУББОТУ

In [408]:
# субботы за каждый месяц
sat_09 = ['03','10','17','24']
sat_10 = ['01','08','15','22','29']
sat_11 = ['05','12','19','26']

### Первые 3 АЗС

In [409]:
# Сбор данных за субботы (сумма кол-ва чеков по часам)
first_sat = pd.read_sql(df(sat_09, sat_10, sat_11, first), connection)
first_sat = format(first_sat)

In [35]:
chart(first_sat['time'], first_sat['amount'], 'Среднее кол-во чеков по часам в субботу', morning(first_sat), day(first_sat))

### Последние 3 АЗС

In [411]:
# Сбор данных за субботы (сумма кол-ва чеков по часам)
last_sat = pd.read_sql(df(sat_09, sat_10, sat_11, last), connection)
last_sat = format(last_sat)

In [36]:
chart(last_sat['time'], last_sat['amount'], 'Среднее кол-во чеков по часам в субботу', morning(last_sat), day(last_sat))

## Аномалии (кол-во чеков) в СУББОТУ

### Первые 3 АЗС

In [413]:
# создание таблицы (день:кол-во чеков:час) по каждой субботе
first_sat_full = pd.read_sql(df_full(sat_09, sat_10, sat_11, first), connection)
first_sat_full = format_2(first_sat_full)

In [37]:
morning_an (first_sat_full)

In [38]:
day_an (first_sat_full)

### Последние 3 АЗС

In [416]:
# создание таблицы (день:кол-во чеков:час) по каждой субботе
last_sat_full = pd.read_sql(df_full(sat_09, sat_10, sat_11, last), connection)
last_sat_full = format_2(last_sat_full)

In [39]:
morning_an (last_sat_full)

In [40]:
day_an (last_sat_full)

# ВОСКРЕСЕНЬЕ

## Среднее кол-во чеков по часам в ВОСКРЕСЕНЬЕ

In [419]:
# воскресенья за каждый месяц
sun_09 = ['04','11','18','25']
sun_10 = ['02','09','16','23','30']
sun_11 = ['06','13','20','27']

### Первые 3 АЗС

In [420]:
# Сбор данных за воскресенья (сумма кол-ва чеков по часам)
first_sun = pd.read_sql(df(sun_09, sun_10, sun_11, first), connection)
first_sun = format(first_sun)

In [41]:
chart(first_sun['time'], first_sun['amount'], 'Среднее кол-во чеков по часам в воскресенье', morning(first_sun), day(first_sun))

### Последние 3 АЗС

In [422]:
# Сбор данных за воскресенья (сумма кол-ва чеков по часам)
last_sun = pd.read_sql(df(sun_09, sun_10, sun_11, last), connection)
last_sun = format(last_sun)

In [42]:
chart(last_sun['time'], last_sun['amount'], 'Среднее кол-во чеков по часам в воскресенье', morning(last_sun), day(last_sun))

## Аномалии (кол-во чеков) в ВОСКРЕСЕНЬЕ

### Первые 3 АЗС

In [424]:
# создание таблицы (день:кол-во чеков:час) по каждому воскресенью
first_sun_full = pd.read_sql(df_full(sun_09, sun_10, sun_11, first), connection)
first_sun_full = format_2(first_sun_full)

In [43]:
morning_an (first_sun_full)

In [44]:
day_an (first_sun_full)

### Последние 3 АЗС

In [427]:
# создание таблицы (день:кол-во чеков:час) по каждому воскресенью
last_sun_full = pd.read_sql(df_full(sun_09, sun_10, sun_11, last), connection)
last_sun_full = format_2(last_sun_full)